<H1>Introduction</H1>

This project will analyze neighborhoods between Toronto, Canada and New York City, New York. A Fortune 500 company is looking to move its headquarters to either Toronto or New York City. The company wants insight into the neighborhoods and local businesses in the cities so that its employees may have the optimum living standards and quality of life. This project will explore the similarities and dissimilarities between certain neighborhoods in the two cities, and determine which neighborhoods best fit the culture of the Fortune 500 company’s employees.

<H3>Segmenting and Clustering Neighborhoods in Toronto</H3>

Use data from Toronto Neighborhood Wikipedia page to segment, cluster and explore neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

<H3>Download and Explore Toronto Dataset</H3>

The dateset being used is found at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

The dataset is a list of Toronto's zipcodes which includes the boroughs and neighborhood names.

In [3]:
#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<H3>Transform the data</H3>

In [6]:
df_toronto.rename(columns = {"Postal Code": "PostalCode", "Neighborhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
#df.head()

#Combine the neighborhoods that exists in one postal code
df_toronto = df_toronto.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
#df.head()

#Change unassigned Neighborhood to its Borough's name
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<H3>Join neighborhood table with latitude and longitude information</H3>

In [7]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
latlong.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
latlong.shape

(103, 3)

In [10]:
latlong.tail()

,PostalCode,Latitude,Longitude
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437
102,M9W,43.706748,-79.594054


<H3>Join latitude and longitude dataframe with neighborhood dataframe</H3>

In [11]:
#Join the Lat and Long dataframe to Neighborhoods dataframe
df_toronto.set_index("PostalCode")
latlong.set_index("PostalCode")
neighbor=pd.merge(df_toronto, latlong)
neighbor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(neighbor['Borough'].unique()),
        neighbor.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


<H3>Use geopy library to get the latitude and longitude values of Toronto, Canada</H3>

In [14]:
address = 'Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


<H3>Create a map of Toronto with neighborhoods superimposed on top</H3>

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Scarborough Neighborhood data.

In [16]:
scarborough_data = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [17]:
scarborough_data.shape

(17, 5)

Lets get the geographical coordinates of Scarborough

In [18]:
address = 'Scarborough, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, CA are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.


In [19]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

<H3>Explore Scarborough neighborhood in Toronto with Foursquare API
    
Define Foursquare credentials and version</H3>

In [20]:
CLIENT_ID = 'D5MZJLGWGSZG2EQ45SVO0A3DB4LEU2UAMFRITACXI4ZA0IDH' # your Foursquare ID
CLIENT_SECRET = 'SD1X35D3L2NZUZDMFA3AIVWRUS50HXAKGCXGUIPY5BCLJU0G' # your Foursquare Secret
VERSION = '20200523' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D5MZJLGWGSZG2EQ45SVO0A3DB4LEU2UAMFRITACXI4ZA0IDH
CLIENT_SECRET:SD1X35D3L2NZUZDMFA3AIVWRUS50HXAKGCXGUIPY5BCLJU0G


Function to explore neighborhoods

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [22]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Found 95 venues in 17 neighborhoods.


In [23]:
print(scarborough_venues.shape)
scarborough_venues.head()

(95, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [24]:
#Venues per Neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",8,8,8,8,8,8


In [25]:
print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

#print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 84 distinct venues in 55 categories.


Analyze each Neighborhood

In [26]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.0,0.250000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.25,0.25,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.250000,0.00,0.0,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.2,0.000,0.000000,0.000000,0.2,0.0,0.000000,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.2,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.200000,0.00,0.0,0.000000,0.0
2,Cedarbrae,0.000000,0.125,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.125,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.125000,0.125000,0.0,0.000000,0.125,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000,0.071429,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.071429,0.071429,0.0,0.0,0.071429,0.000,0.000000,0.000000,0.0,0.071429,0.071429,0.071429,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.071429,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.071429,0.0,0.0,0.071429,0.142857,0.0,0.0,0.000,0.000000,0.071429,0.000000,0.00,0.0,0.071429,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.333333,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.333333,0.00,0.0,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.200000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.40,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.2,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.200,0.000000,0.0,0.000000,0.000000,0.2,0.100,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.1,0.00,0.100,0.000000,0.00,0.00,0.00,0.000,0.1,0.000,0.00,0.000000,0.000,0.000000,0.1,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000,0.125000,0.0,0.125000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.125,0.000000,0.00,0.00,0.00,0.125,0.0,0.125,0.00,0.000000,0.125,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.125,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0
8,"Kennedy Park, Ionview

Each neighborhood with the top 5 venues

In [28]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4        American Restaurant  0.00
5                       Park  0.00
6         Italian Restaurant  0.00
7          Korean Restaurant  0.00
8             Medical Center  0.00
9              Metro Station  0.00


----Birch Cliff, Cliffside West----
                       venue  freq
0               Skating Rink   0.2
1      General Entertainment   0.2
2                       Café   0.2
3                       Farm   0.2
4            College Stadium   0.2
5        American Restaurant   0.0
6                       Park   0.0
7  Latin American Restaurant   0.0
8                     Lounge   0.0
9             Medical Center   0.0


----Cedarbrae----
                       venue  freq
0                     Bakery  0.12
1                       Bank  0.12
2            Thai Restaurant  0.12
3        Fried

Let's put that into a pandas dataframe

Let's write a function to sort the venues in descending order

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top ten venues for each neighborhood

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
7,"Guildwood, Morningside, West Hill",Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
8,"Kennedy Park, Ionview, East Birchmount Park",Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
9,"Malvern, Rouge",Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store


In [31]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Rouge Hill, Port Union, Highland Creek
1st Most Common Venue                              Moving Target
2nd Most Common Venue                                        Bar
3rd Most Common Venue                      Vietnamese Restaurant
4th Most Common Venue                              Grocery Store
5th Most Common Venue                      General Entertainment
6th Most Common Venue                                Gas Station
7th Most Common Venue                        Fried Chicken Joint
8th Most Common Venue                       Fast Food Restaurant
9th Most Common Venue                                       Farm
10th Most Common Venue                         Electronics Store
Name: 11, dtype: object

<H3>4. Cluster the Scarborough Neighborhood using k-means</H3>
Run K-means to cluster neighborhood into three clusters

In [32]:
# set number of clusters
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0], dtype=int32)

Lets create a new dataframe that includes the cluster as well as the top ten venues for each neighborhood

In [33]:
#Note that the neighborhood Upper Rouge does not have any venues, so I will drop from dataset
scarborough_data.drop(scarborough_data[scarborough_data.Neighborhood == 'Upper Rouge'].index, inplace = True)
#df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)

scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<H3>5. Examine Scarborough Neighborhood Cluster</H3>
Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.
Based on the defining categories, you can then assign a name to each cluster.

Scarborough Clusters 0, 1, 2

In [35]:
scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [36]:
scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,-79.194353,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store
1,Scarborough,-79.160497,0,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
2,Scarborough,-79.188711,0,Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
3,Scarborough,-79.216917,0,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
4,Scarborough,-79.239476,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
5,Scarborough,-79.239476,0,Playground,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
6,Scarborough,-79.262029,0,Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
7,Scarborough,-79.284577,0,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
8,Scarborough,-79.239476,0,American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
9,Scarborough,-79.264848,0,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [37]:
scarborough_cluster_1

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Scarborough,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
12,Scarborough,-79.262029,1,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [38]:
scarborough_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,-79.295849,2,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


<H3>6. Explore New York City Neighborhoods</H3>

New York City has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Luckily, this dataset exists for free on the web. The link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For convenience, I downloaded the files and placed it on the server, so we can simply run a wget command and access the data. So let's go ahead and do that.

In [39]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


Load and explore dataset

In [40]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [41]:
neighborhoods_data = newyork_data['features']

In [42]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Transform the data into a pandas dataframe

In [43]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [44]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [45]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [46]:
neighborhoods.shape

(306, 4)

In [47]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Use Geolibrary to get the latitude and longitude of New York City

In [48]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [49]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Lets simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Queens neighborhood data.

In [50]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


Lets get the geographical location of Queens, NY

In [51]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [52]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

Lets explore the Long Island City neighborhood in Queens, NY

In [53]:
queens_data.loc[10, 'Neighborhood']

'Long Island City'

In [54]:
#Long Island City Latitude and Longitude values

neighborhood_latitude = queens_data.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queens_data.loc[10, 'Longitude'] # neighborhood longitude value

neighborhood_name = queens_data.loc[10, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Long Island City are 40.75021734610528, -73.93920223915505.


Top 100 venues in Long Island City neighborhood within a radius of 500 meters
First, let's create the GET request URL named url.

In [55]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=D5MZJLGWGSZG2EQ45SVO0A3DB4LEU2UAMFRITACXI4ZA0IDH&client_secret=SD1X35D3L2NZUZDMFA3AIVWRUS50HXAKGCXGUIPY5BCLJU0G&v=20200523&ll=40.75021734610528,-73.93920223915505&radius=500&limit=100'

In [56]:
#Send the GET request
results = requests.get(url).json()

In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [58]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Chip,Dessert Shop,40.750069,-73.940831
1,Etto Espresso Bar,Coffee Shop,40.748703,-73.940689
2,Hilton Garden Inn,Hotel,40.750216,-73.936886
3,Dutch Kills,Cocktail Bar,40.747830,-73.940108
4,Brooklyn Boulders Queensbridge,Climbing Gym,40.752649,-73.940010


In [59]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

72 venues were returned by Foursquare.


<H3>7. Analyze Each Neighborhood in Queens</H3>

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [ ]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude'])

In [ ]:
queens_venues.head()

In [ ]:
print(queens_venues.shape)
queens_venues.tail()

In [ ]:
#Venues per Neighborhood
queens_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} distinct venues in {} categories.'.format(
    len(queens_venues['Venue'].unique()),len(queens_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
#scarborough_onehot = scarborough_onehot[fixed_columns]

neighbor = queens_onehot['Neighborhood']
queens_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
queens_onehot.insert(0, 'Neighborhood', neighbor)

queens_onehot.head()

Group by Neighborhood, and exame the frequency of the occurrence of venue

In [ ]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

Each Neighborhood with the top 5 venues

In [ ]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Top venues for each neighborhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
neighborhoods_venues_sorted.iloc[47,]

Cluster the Queens Borough using K-Means

In [ ]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

Dataframe that includes the cluster of each neighborhood

In [ ]:
queens_merged = queens_data

# add clustering labels
queens_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<H3>Examine Queens Cluster Neighborhood</H3>

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

In [ ]:
queens_cluster_0 = queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_1 = queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_2 = queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_3 = queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

queens_cluster_4 = queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(4, queens_merged.shape[1]))]]

In [ ]:
queens_cluster_0

In [ ]:
queens_cluster_1

In [ ]:
queens_cluster_2

In [ ]:
queens_cluster_3

In [ ]:
queens_cluster_4